In [71]:
import pandas as pd
import numpy as np

Defino los path

In [72]:
path = '/home/oblitterator/00 REFEPS/Data_Wrangling/df_limpios/20220408_df_matriculas_limpio.csv'

In [73]:
path_matriculas_refeps = 'home/oblitterator/00 REFEPS/Data_Wrangling/df_sucios/Bajada Matriculas 08042022.txt'
fecha_query = path_matriculas_refeps[-13:-4] #Calculo la fecha en base al path

In [74]:
fecha_query

' 08042022'

In [75]:
path_sintys_personas = '/home/oblitterator/0 SINTyS/REFEPS_recibido/REFEPS_020222/para_cruce_REFEPS/id_persona_refeps0222.csv'
path_sintys_fecha_nac = '/home/oblitterator/0 SINTyS/REFEPS_recibido/REFEPS_020222/para_cruce_REFEPS/fechas_nacimiento0222.csv'
path_sintys_fallecidos = '/home/oblitterator/0 SINTyS/REFEPS_recibido/REFEPS_020222/para_cruce_REFEPS/fallecidos0222.csv'
path_sintys_domicilios = '/home/oblitterator/0 SINTyS/REFEPS_recibido/REFEPS_020222/para_cruce_REFEPS/domicilios0222.csv'

In [76]:
## Fecha de corte para cálculo

#Conformo la fecha para después calcular las edades
fecha = pd.to_datetime('2020-30-06', #imputo la fecha manualmente sino colocar fecha_query
                      yearfirst=False,
                      dayfirst=True,
                      infer_datetime_format=True)

# Levanto el DF de Matrículas

In [77]:
df = pd.read_csv(path,
                 encoding= 'latin')

/home/oblitterator/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (14,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1523954 entries, 0 to 1523953
Data columns (total 35 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID_PROFESIONAL            1523954 non-null  int64  
 1   CODIGO                    1523954 non-null  int64  
 2   NOMBRE                    1523954 non-null  object 
 3   APELLIDO                  1523953 non-null  object 
 4   ID_TIPODOC                1523954 non-null  int64  
 5   NRODOC                    1523954 non-null  int64  
 6   FECHA_NACIMIENTO          1521518 non-null  object 
 7   ID_PAIS_NACIMIENTO        1523954 non-null  int64  
 8   SEXO                      1523954 non-null  object 
 9   CUIL                      1523954 non-null  int64  
 10  ID_PROVINCIA_RENAPER      1523954 non-null  int64  
 11  PROVINCIA_CMDB            1159984 non-null  object 
 12  ID_CMDB_PERSONA           1523954 non-null  int64  
 13  FALLECIDO                 1

In [79]:
#elimino los que no tienen fecha de matriculacion

df = df[df['FECHA_MATRICULA'].notnull()]

In [80]:
#convierto las fechas de matriculación a tipo datetime

df['FECHA_MATRICULA'] = pd.to_datetime(df['FECHA_MATRICULA'])

# Levanto los DF de Fallecidos y Domicilios

In [81]:
df_fallecidos = pd.read_csv(path_sintys_fallecidos,
                 encoding= 'latin')

In [82]:
df_fallecidos['FALLECIDO'] = True

In [83]:
df_domicilios = pd.read_csv(path_sintys_domicilios,
                 encoding= 'latin')

df_domicilios = df_domicilios[['ID_PROFESIONAL','CODIGO_POSTAL',
       'LOCALIDAD', 'DEPARTAMENTO', 'PROVINCIA']]

/home/oblitterator/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (6,7,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [84]:
df_fecha_nac = pd.read_csv(path_sintys_fecha_nac,
                 encoding= 'latin')

# Imputo fallecidos

In [85]:
df = pd.merge(df, df_fallecidos, on = "ID_PROFESIONAL", how = 'left')

In [86]:
#df = df[(df['FALLECIDO_y'] != 'SI' ) & (df['FALLECIDO_x'] != 'SI')]

In [87]:
df['FALLECIDO'] = np.nan

In [88]:
df['FALLECIDO_x'].value_counts()

False    1425639
True       97922
Name: FALLECIDO_x, dtype: int64

In [89]:
df['FALLECIDO_y'].value_counts()

True    96480
Name: FALLECIDO_y, dtype: int64

In [90]:
df['FALLECIDO'] = df['FALLECIDO'].fillna(df['FALLECIDO_x']).fillna(df['FALLECIDO_y'])

In [91]:
df['FALLECIDO'].value_counts()

False    1425639
True       97922
Name: FALLECIDO, dtype: int64

In [92]:
df['FECHA_FALLECIDO'].isnull().value_counts()

True     1431233
False      92328
Name: FECHA_FALLECIDO, dtype: int64

In [93]:
df['FECHA_FALLECIDO'].fillna(df['FFALL'], inplace = True)

In [94]:
df['FECHA_FALLECIDO'].isnull().value_counts()

True     1431229
False      92332
Name: FECHA_FALLECIDO, dtype: int64

In [95]:
df = df.drop(['Unnamed: 0',
              'FFALL',
              'FALLECIDO_x',
              'FALLECIDO_y'], axis = 1)

In [96]:
#remuevo valores faltantes de fallecidos sin fecha

df = df.drop(df[(df["FECHA_FALLECIDO"].isnull()) & (df["FALLECIDO"]== True)].index)

In [97]:
df.loc[df['FECHA_FALLECIDO'] == '201-10-18']

,ID_PROFESIONAL,CODIGO,NOMBRE,APELLIDO,ID_TIPODOC,NRODOC,FECHA_NACIMIENTO,ID_PAIS_NACIMIENTO,SEXO,CUIL,...,PROVINCIA_MATRICULA,ID_SITUACION_MATRICULA,COLEGIO_REGISTRA,ACTIVO,COLEGIO_EMITE,ID_ORIGEN_EMITE,ORIGEN_EMITE,ID_PROFESIONAL_SANCION,EDAD,FALLECIDO


In [98]:
#convierto a fecha con coerce (analizar variante para trabajar con datos e imputaciones)


df["FECHA_FALLECIDO"] = pd.to_datetime(df["FECHA_FALLECIDO"], errors = 'coerce')

# Imputo provincias

In [99]:
df['prov_adjudicada'] = np.nan

In [100]:
df = pd.merge(df, df_domicilios, on = "ID_PROFESIONAL", how = 'left')

In [101]:
df['PROVINCIA'].unique()

array(['BUENOS AIRES', 'SANTA CRUZ', 'CIUDAD AUTONOMA DE BS AS',
       'CORDOBA', 'TIERRA DEL FUEGO', 'CHUBUT', 'NEUQUEN', 'CATAMARCA',
       'RIO NEGRO', 'MISIONES', 'ENTRE RIOS', nan, 'MENDOZA', 'SAN LUIS',
       'SANTA FE', 'FORMOSA', 'LA PAMPA', 'SAN JUAN', 'JUJUY', 'SALTA',
       'SANTIAGO DEL ESTERO', 'CORRIENTES', 'LA RIOJA', 'CHACO',
       'TUCUMAN'], dtype=object)

In [102]:
 cod_prov = {'CORDOBA' :6 , 'CIUDAD AUTONOMA DE BS AS' : 1, 'SAN JUAN': 18, 'RIO NEGRO' : 16,
       'NEUQUEN' : 15 , 'TIERRA DEL FUEGO' : 23, 'BUENOS AIRES': 2, 'TUCUMAN': 24, 'CHACO': 4,
       'ENTRE RIOS' : 8, 'MENDOZA' : 13, 'CORRIENTES' : 7, 'LA RIOJA' : 12, 'SANTA FE' : 21,
       'SAN LUIS' :19, 'FORMOSA' : 9, 'CATAMARCA' : 3 , 'SALTA' : 17 , 'MISIONES' :14, 'CHUBUT':5,
       'SANTA CRUZ' : 20, 'LA PAMPA' : 11, 'SANTIAGO DEL ESTERO' : 22, 'JUJUY':10}

In [103]:
df['PROVINCIA'] = df['PROVINCIA'].map(cod_prov)

In [104]:
df.columns

Index(['ID_PROFESIONAL', 'CODIGO', 'NOMBRE', 'APELLIDO', 'ID_TIPODOC',
       'NRODOC', 'FECHA_NACIMIENTO', 'ID_PAIS_NACIMIENTO', 'SEXO', 'CUIL',
       'ID_PROVINCIA_RENAPER', 'PROVINCIA_CMDB', 'ID_CMDB_PERSONA',
       'FECHA_FALLECIDO', 'ID_PROFESIONAL_PROFESION',
       'ID_PROFESION_REFERENCIA', 'PROFESION_REFERENCIA',
       'ID_INSTITUCION_FORMADORA', 'DENOMINACION', 'FECHA_TITULO',
       'ID_PROFESIONAL_MATRICULA', 'MATRICULA', 'FECHA_MATRICULA',
       'REMATRICULACION', 'ID_PROVINCIA_MATRICULA', 'PROVINCIA_MATRICULA',
       'ID_SITUACION_MATRICULA', 'COLEGIO_REGISTRA', 'ACTIVO', 'COLEGIO_EMITE',
       'ID_ORIGEN_EMITE', 'ORIGEN_EMITE', 'ID_PROFESIONAL_SANCION', 'EDAD',
       'FALLECIDO', 'prov_adjudicada', 'CODIGO_POSTAL', 'LOCALIDAD',
       'DEPARTAMENTO', 'PROVINCIA'],
      dtype='object')

In [105]:
df['prov_adjudicada'] = df['prov_adjudicada'].fillna(df['PROVINCIA'])

In [106]:
df.columns

Index(['ID_PROFESIONAL', 'CODIGO', 'NOMBRE', 'APELLIDO', 'ID_TIPODOC',
       'NRODOC', 'FECHA_NACIMIENTO', 'ID_PAIS_NACIMIENTO', 'SEXO', 'CUIL',
       'ID_PROVINCIA_RENAPER', 'PROVINCIA_CMDB', 'ID_CMDB_PERSONA',
       'FECHA_FALLECIDO', 'ID_PROFESIONAL_PROFESION',
       'ID_PROFESION_REFERENCIA', 'PROFESION_REFERENCIA',
       'ID_INSTITUCION_FORMADORA', 'DENOMINACION', 'FECHA_TITULO',
       'ID_PROFESIONAL_MATRICULA', 'MATRICULA', 'FECHA_MATRICULA',
       'REMATRICULACION', 'ID_PROVINCIA_MATRICULA', 'PROVINCIA_MATRICULA',
       'ID_SITUACION_MATRICULA', 'COLEGIO_REGISTRA', 'ACTIVO', 'COLEGIO_EMITE',
       'ID_ORIGEN_EMITE', 'ORIGEN_EMITE', 'ID_PROFESIONAL_SANCION', 'EDAD',
       'FALLECIDO', 'prov_adjudicada', 'CODIGO_POSTAL', 'LOCALIDAD',
       'DEPARTAMENTO', 'PROVINCIA'],
      dtype='object')

In [107]:
df['prov_adjudicada']  = df['prov_adjudicada'].fillna(df['ID_PROVINCIA_RENAPER']).fillna(df['PROVINCIA_MATRICULA'])

In [108]:
df['prov_adjudicada'].isnull().value_counts()

False    1516553
Name: prov_adjudicada, dtype: int64

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1516553 entries, 0 to 1516552
Data columns (total 40 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   ID_PROFESIONAL            1516553 non-null  int64         
 1   CODIGO                    1516553 non-null  int64         
 2   NOMBRE                    1516553 non-null  object        
 3   APELLIDO                  1516552 non-null  object        
 4   ID_TIPODOC                1516553 non-null  int64         
 5   NRODOC                    1516553 non-null  int64         
 6   FECHA_NACIMIENTO          1514121 non-null  object        
 7   ID_PAIS_NACIMIENTO        1516553 non-null  int64         
 8   SEXO                      1516553 non-null  object        
 9   CUIL                      1516553 non-null  int64         
 10  ID_PROVINCIA_RENAPER      1516553 non-null  int64         
 11  PROVINCIA_CMDB            1156936 non-null  object

# Imputo fechas de nacimiento

In [110]:
df = pd.merge(df, df_fecha_nac, on = "ID_PROFESIONAL", how = 'left')

In [111]:
df['FECHA_NACIMIENTO'].isnull().value_counts()

False    1514121
True        2432
Name: FECHA_NACIMIENTO, dtype: int64

In [112]:
df['FECHA_NACIMIENTO'].fillna(df['fnac'], inplace = True)

In [113]:
df['FECHA_NACIMIENTO'].isnull().value_counts()

False    1514121
True        2432
Name: FECHA_NACIMIENTO, dtype: int64

In [114]:
df.columns

Index(['ID_PROFESIONAL', 'CODIGO', 'NOMBRE', 'APELLIDO', 'ID_TIPODOC',
       'NRODOC', 'FECHA_NACIMIENTO', 'ID_PAIS_NACIMIENTO', 'SEXO', 'CUIL',
       'ID_PROVINCIA_RENAPER', 'PROVINCIA_CMDB', 'ID_CMDB_PERSONA',
       'FECHA_FALLECIDO', 'ID_PROFESIONAL_PROFESION',
       'ID_PROFESION_REFERENCIA', 'PROFESION_REFERENCIA',
       'ID_INSTITUCION_FORMADORA', 'DENOMINACION', 'FECHA_TITULO',
       'ID_PROFESIONAL_MATRICULA', 'MATRICULA', 'FECHA_MATRICULA',
       'REMATRICULACION', 'ID_PROVINCIA_MATRICULA', 'PROVINCIA_MATRICULA',
       'ID_SITUACION_MATRICULA', 'COLEGIO_REGISTRA', 'ACTIVO', 'COLEGIO_EMITE',
       'ID_ORIGEN_EMITE', 'ORIGEN_EMITE', 'ID_PROFESIONAL_SANCION', 'EDAD',
       'FALLECIDO', 'prov_adjudicada', 'CODIGO_POSTAL', 'LOCALIDAD',
       'DEPARTAMENTO', 'PROVINCIA', 'Unnamed: 0', 'fnac'],
      dtype='object')

In [115]:
df.drop(['fnac', 'EDAD'], axis = 1, inplace = True) #'Unnamed: 0_x', 'Unnamed: 0_y',

# Enfermería por Máximo Grado

Creo un DF de apoyo para definir máximo grado alcanzado por ID_PROFESIONAL


In [116]:
df_max_grado = df.loc[(df['ID_PROFESION_REFERENCIA']==15)|
                      (df['ID_PROFESION_REFERENCIA']==32)|
                      (df['ID_PROFESION_REFERENCIA']==37)]

In [117]:
df_max_grado['enf_max_grado'] = np.nan
df_max_grado['enf_max_grado'] = df_max_grado['enf_max_grado'].fillna(df_max_grado['ID_PROFESION_REFERENCIA']).astype(int)

/tmp/ipykernel_7328/1948086837.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_max_grado['enf_max_grado'] = np.nan
/tmp/ipykernel_7328/1948086837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_max_grado['enf_max_grado'] =df_max_grado['enf_max_grado'].fillna(df_max_grado['ID_PROFESION_REFERENCIA']).astype(int)


In [118]:
enf_map = {37:1, 15:2, 32: 3}
df_max_grado['enf_max_grado'].replace(enf_map, inplace = True)

/home/oblitterator/.local/lib/python3.8/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [119]:
df_max_grado = df_max_grado.sort_values('enf_max_grado').drop_duplicates('ID_PROFESIONAL')

In [120]:
df_max_grado = df_max_grado[['ID_PROFESIONAL', 'enf_max_grado', 'fecha_enf_max_grado']]

In [121]:
df_max_grado
pam_fne = {1:37, 2:15, 3: 32}
df_max_grado['enf_max_grado'].replace(pam_fne, inplace = True)

Combino la columna creada, donde indico el máximo nivel, con el DF original para cada ID_PROFESIONAL

In [122]:
df = pd.merge(df,df_max_grado, on = 'ID_PROFESIONAL', how = 'left')

In [123]:
df.loc[(df['ID_PROFESION_REFERENCIA']==15)|
                      (df['ID_PROFESION_REFERENCIA']==32)|
                      (df['ID_PROFESION_REFERENCIA']==37)][['ID_PROFESION_REFERENCIA', 'enf_max_grado']]

,ID_PROFESION_REFERENCIA,enf_max_grado
2675,32,15.0
2676,15,15.0
3280,15,15.0
3655,32,32.0
3771,32,32.0
...,...,...
1516512,37,37.0
1516513,32,15.0
1516514,32,15.0
1516515,15,15.0


In [124]:
#Tengo que trabajar en criterio de última matriculación para adjudicar profesión de referencia,
#porque sino estoy sobredimensionando los enfermeros

# Imputación de profesión

In [125]:
df['prof_adjudicada'] = np.nan

In [126]:
df['prof_adjudicada'].fillna(df['enf_max_grado'], inplace = True)

In [127]:
df['prof_adjudicada'].value_counts()

15.0    247716
32.0    124109
37.0    122984
Name: prof_adjudicada, dtype: int64

In [128]:
df['prof_adjudicada'] = df['prof_adjudicada'].fillna(df['ID_PROFESION_REFERENCIA'])

In [129]:
df['prof_adjudicada'].value_counts()

35.0     442737
15.0     247716
1.0      135880
32.0     124109
37.0     122984
          ...  
178.0         4
183.0         3
105.0         3
126.0         3
43.0          1
Name: prof_adjudicada, Length: 82, dtype: int64

In [130]:
df['ACTIVO'].value_counts()

True     1400571
False     115982
Name: ACTIVO, dtype: int64

In [131]:
df['FALLECIDO'].value_counts()

False    1425639
True       90914
Name: FALLECIDO, dtype: int64

In [132]:
df['ID_SITUACION_MATRICULA'].value_counts()

1    1366553
3     127555
2      14861
4       7581
0          3
Name: ID_SITUACION_MATRICULA, dtype: int64

# Exporto DF con cruces de SINTyS para cálculos de totales

In [ ]:
df.to_csv('Profesionales/matriculas_con_cruce_sintys.csv', encoding = 'latin')


In [ ]:
df.columns

## Armo DF reducido para Dashboard

In [ ]:
cols_dashboard = ['ID_PROFESIONAL', 'FECHA_NACIMIENTO', 'ID_PAIS_NACIMIENTO', 'SEXO', 
       'FALLECIDO', 'FECHA_FALLECIDO', 'FECHA_TITULO', 'FECHA_MATRICULA',
       'ID_SITUACION_MATRICULA', 'ACTIVO', 'COLEGIO_REGISTRA',
       'ID_ORIGEN_EMITE', 'ID_PROFESIONAL_SANCION',
       'prov_adjudicada', 'prof_adjudicada']

In [ ]:
df_dashboard = df[cols_dashboard]

In [ ]:
#Ejemplo de info para dashboard

df_dashboard.loc[(df_dashboard['prof_adjudicada'] == 35) |
                 ( df_dashboard['prof_adjudicada'] == 15)].sort_values(
    'FECHA_MATRICULA', ascending = False).drop_duplicates('ID_PROFESIONAL').to_csv('df_ejemplo_dash.csv')

## Separación DF CABA con cruce SINTyS

In [ ]:
df_matriculas_caba = (df.loc[df['PROVINCIA_MATRICULA'] == 'CABA'])
df_matriculas_caba.to_csv('matriculas_caba.csv', encoding = 'latin')

In [ ]:
df_matriculas_caba

# Armo otros DF para cuentas generales

## Saco los matriculados después de la fecha

In [ ]:
df = df.loc[df['FECHA_MATRICULA']<=fecha]

## Calculo las edades

In [ ]:
def age(birth_date):
    today = fecha
    y = today.year - birth_date.year
    if today.month < birth_date.month or today.month == birth_date.month and today.day < birth_date.day:
        y -= 1
    return y

In [ ]:
pd.to_datetime(df['FECHA_NACIMIENTO'])

In [ ]:
df['FECHA_NACIMIENTO'] = pd.to_datetime(df['FECHA_NACIMIENTO'])
df['EDAD'] = df['FECHA_NACIMIENTO'].apply(age)

In [ ]:
df['EDAD'].isnull().value_counts()

In [ ]:
df['EDAD'] = df['EDAD'].fillna(99).astype(int)

# Determino los no fallecidos a la fecha de corte

In [ ]:
df['fallecido_al_corte'] = df['FECHA_FALLECIDO'] < fecha

In [ ]:
df['fallecido_al_corte'].value_counts()

In [ ]:
df['año_corte'] = fecha.year

# Selecciono matriculados antes de la fecha de corte

In [ ]:
df = df.loc[df['FECHA_MATRICULA'] <= fecha]

In [ ]:
df.to_csv('df_sin_fallecidos_al'+str(fecha.year)+ '.csv', encoding = 'latin')